Divide larger chunks into 2-second equivalents so that the model gets the same size data.

In [194]:
import os
import pandas as pd
from collections import OrderedDict
import ast
import numpy as np

In [218]:
label = "back_2_seconds"

In [219]:
file_path = f"prepared_data/{label}.csv"

In [220]:
df = pd.read_csv(file_path)
df

,Unnamed: 0,Category,Accelerometer,Gyroscope,Magnetometer,Duration
0,0,CLIMBING,"[[-6.850950241088867, 6.60105037689209, -1.287...","[[0.29122498631477356, 0.5505499839782715, 0.2...","[[65.8125, -19.012500762939453, 16.80000114440...",2
1,1,CLIMBING,"[[-7.216950416564941, 3.28695011138916, -5.781...","[[0.7684875130653381, 1.5860625505447388, -0.0...","[[63.656253814697266, -6.656250476837158, 22.5...",2
2,2,CLIMBING,"[[-5.070000171661377, 5.26200008392334, -3.694...","[[-0.36918750405311584, -0.17173749208450317, ...","[[46.743751525878906, 13.387500762939453, 40.5...",2
3,3,CLIMBING,"[[-5.767050266265869, 3.8140501976013184, -5.8...","[[0.503250002861023, 0.7369999885559082, 0.188...","[[39.1875, -6.0, 47.10000228881836], [41.28750...",2
4,4,CLIMBING,"[[-7.998000144958496, 3.079050064086914, -4.67...","[[0.6491374969482422, 0.3961375057697296, 0.07...","[[57.187503814697266, 4.087500095367432, 25.50...",2
...,...,...,...,...,...,...
348,348,WALKING,"[[-6.4009504318237305, 6.340050220489502, -0.3...","[[1.5175875425338745, -1.1064624786376953, -0....","[[31.912500381469727, -32.10000228881836, 17.2...",2
349,349,WALKING,"[[-4.579050064086914, 8.326950073242188, -0.24...","[[-0.2815999984741211, 0.30607500672340393, -0...","[[37.14375305175781, -27.187501907348633, 10.5...",2
350,350,WALKING,"[[-6.3510003089904785, 5.775000095367432, -0.3...","[[-0.06228749826550484, 0.14726249873638153, 0...","[[26.287500381469727, -35.400001525878906, -20...",2
351,351,WALKING,"[[-7.014000415802002, 8.842050552368164, -0.07...","[[0.1326874941587448, -0.2755500078201294, -0....","[[29.887500762939453, -30.95625114440918, -21....",2


In [221]:
def convert_to_float_list(string_list):
    nested_list = ast.literal_eval(string_list)
    float_list = [[float(item) for item in sublist] for sublist in nested_list]
    return float_list

In [222]:
df['Accelerometer'] = df['Accelerometer'].apply(convert_to_float_list)
df['Gyroscope'] = df['Gyroscope'].apply(convert_to_float_list)
df['Magnetometer'] = df['Magnetometer'].apply(convert_to_float_list)

In [223]:
new_df = pd.DataFrame(columns=df.columns)
# duration in seconds
new_duration = 2
acc_gyro_value_cap = new_duration*50
mag_value_cap = new_duration*5

for index, row in df.iterrows():
    accelerometer_data = row['Accelerometer']
    gyroscope_data = row['Gyroscope']
    magnetometer_data = row['Magnetometer']

    # Podział danych z akcelerometru i żyroskopu na bloki
    accelerometer_blocks = [accelerometer_data[i:i + acc_gyro_value_cap] for i in range(0, len(accelerometer_data), acc_gyro_value_cap)]
    gyroscope_blocks = [gyroscope_data[i:i + acc_gyro_value_cap] for i in range(0, len(gyroscope_data), acc_gyro_value_cap)]

    # Podział danych z magnetometru na bloki
    magnetometer_blocks = [magnetometer_data[i:i + mag_value_cap] for i in range(0, len(magnetometer_data), mag_value_cap)]

    # Dodawanie przekonwertowanych danych do nowego dataframe'u
    min_blocks = min(len(accelerometer_blocks), len(gyroscope_blocks), len(magnetometer_blocks))
    for i in range(min_blocks):
        new_row = {
            'Category': row["Category"],
            'Accelerometer': accelerometer_blocks[i],
            'Gyroscope': gyroscope_blocks[i],
            'Magnetometer': magnetometer_blocks[i],
            'Duration': new_duration
        }
        new_df.loc[len(new_df.index)] = new_row


In [224]:
new_df = new_df.drop(columns=['Unnamed: 0'])
new_df

,Category,Accelerometer,Gyroscope,Magnetometer,Duration
0,CLIMBING,"[[-6.850950241088867, 6.60105037689209, -1.287...","[[0.29122498631477356, 0.5505499839782715, 0.2...","[[65.8125, -19.012500762939453, 16.80000114440...",2
1,CLIMBING,"[[-7.216950416564941, 3.28695011138916, -5.781...","[[0.7684875130653381, 1.5860625505447388, -0.0...","[[63.656253814697266, -6.656250476837158, 22.5...",2
2,CLIMBING,"[[-5.070000171661377, 5.26200008392334, -3.694...","[[-0.36918750405311584, -0.17173749208450317, ...","[[46.743751525878906, 13.387500762939453, 40.5...",2
3,CLIMBING,"[[-5.767050266265869, 3.8140501976013184, -5.8...","[[0.503250002861023, 0.7369999885559082, 0.188...","[[39.1875, -6.0, 47.10000228881836], [41.28750...",2
4,CLIMBING,"[[-7.998000144958496, 3.079050064086914, -4.67...","[[0.6491374969482422, 0.3961375057697296, 0.07...","[[57.187503814697266, 4.087500095367432, 25.50...",2
...,...,...,...,...,...
348,WALKING,"[[-6.4009504318237305, 6.340050220489502, -0.3...","[[1.5175875425338745, -1.1064624786376953, -0....","[[31.912500381469727, -32.10000228881836, 17.2...",2
349,WALKING,"[[-4.579050064086914, 8.326950073242188, -0.24...","[[-0.2815999984741211, 0.30607500672340393, -0...","[[37.14375305175781, -27.187501907348633, 10.5...",2
350,WALKING,"[[-6.3510003089904785, 5.775000095367432, -0.3...","[[-0.06228749826550484, 0.14726249873638153, 0...","[[26.287500381469727, -35.400001525878906, -20...",2
351,WALKING,"[[-7.014000415802002, 8.842050552368164, -0.07...","[[0.1326874941587448, -0.2755500078201294, -0....","[[29.887500762939453, -30.95625114440918, -21....",2


In [203]:
new_df.to_csv(f'prepared_data/back_{new_duration}_seconds.csv')

In [204]:
# Because the Gyroscope and Magnetometer values has the lowest impact on the actual research, 
# we drop these values so the trained model will work better
new_df = new_df.drop('Gyroscope', axis=1)
new_df = new_df.drop('Magnetometer', axis=1)

In [225]:
new_df

,Category,Accelerometer,Gyroscope,Magnetometer,Duration
0,CLIMBING,"[[-6.850950241088867, 6.60105037689209, -1.287...","[[0.29122498631477356, 0.5505499839782715, 0.2...","[[65.8125, -19.012500762939453, 16.80000114440...",2
1,CLIMBING,"[[-7.216950416564941, 3.28695011138916, -5.781...","[[0.7684875130653381, 1.5860625505447388, -0.0...","[[63.656253814697266, -6.656250476837158, 22.5...",2
2,CLIMBING,"[[-5.070000171661377, 5.26200008392334, -3.694...","[[-0.36918750405311584, -0.17173749208450317, ...","[[46.743751525878906, 13.387500762939453, 40.5...",2
3,CLIMBING,"[[-5.767050266265869, 3.8140501976013184, -5.8...","[[0.503250002861023, 0.7369999885559082, 0.188...","[[39.1875, -6.0, 47.10000228881836], [41.28750...",2
4,CLIMBING,"[[-7.998000144958496, 3.079050064086914, -4.67...","[[0.6491374969482422, 0.3961375057697296, 0.07...","[[57.187503814697266, 4.087500095367432, 25.50...",2
...,...,...,...,...,...
348,WALKING,"[[-6.4009504318237305, 6.340050220489502, -0.3...","[[1.5175875425338745, -1.1064624786376953, -0....","[[31.912500381469727, -32.10000228881836, 17.2...",2
349,WALKING,"[[-4.579050064086914, 8.326950073242188, -0.24...","[[-0.2815999984741211, 0.30607500672340393, -0...","[[37.14375305175781, -27.187501907348633, 10.5...",2
350,WALKING,"[[-6.3510003089904785, 5.775000095367432, -0.3...","[[-0.06228749826550484, 0.14726249873638153, 0...","[[26.287500381469727, -35.400001525878906, -20...",2
351,WALKING,"[[-7.014000415802002, 8.842050552368164, -0.07...","[[0.1326874941587448, -0.2755500078201294, -0....","[[29.887500762939453, -30.95625114440918, -21....",2


In [226]:
def count_rows_by_category(df):
    category_counts = df['Category'].value_counts()
    values = []
    
    for category, count in category_counts.items():
        print(f"Liczba wierszy dla kategorii '{category}': {count}")
        values.append(count)
    minimum =min(values)
    print(f"Najmniejsza wartosc: {minimum}")
    return minimum


In [227]:
minimum = count_rows_by_category(new_df)

Liczba wierszy dla kategorii 'CLIMBING': 137
Liczba wierszy dla kategorii 'WALKING': 104
Liczba wierszy dla kategorii 'FALLING': 59
Liczba wierszy dla kategorii 'DOWNTIME': 53
Najmniejsza wartosc: 53


In [228]:
def sort_rows_by_mean_z(data_frame):
    # Tworzenie nowej kolumny 'Mean_Z', która zawiera średnie wartości z
    data_frame['Mean_Z'] = data_frame['Accelerometer'].apply(lambda x: sum([i[2] for i in x]) / len(x))

    # Sortowanie DataFrame'u według kolumny 'Mean_Z' w kolejności malejącej
    sorted_df = data_frame.sort_values(by='Mean_Z', ascending=False)

    # Usunięcie kolumny 'Mean_Z' z DataFrame'u
    sorted_df.drop('Mean_Z', axis=1, inplace=True)

    return sorted_df

In [229]:
sorted_df = sort_rows_by_mean_z(new_df)
sorted_df

,Category,Accelerometer,Gyroscope,Magnetometer,Duration
136,CLIMBING,"[[0.7330500483512878, 0.4930500090122223, 10.0...","[[0.5670499801635742, -0.09542500227689743, 0....","[[-129.78750610351562, 333.7125244140625, 301....",2
135,CLIMBING,"[[-0.3169500231742859, 1.4239500761032104, 12....","[[0.1942874938249588, -0.2116124927997589, 0.2...","[[39.84375, 63.95625305175781, 10.612500190734...",2
133,CLIMBING,"[[0.3540000021457672, 3.427950143814087, 9.400...","[[-0.9094250202178955, -2.147200107574463, -0....","[[32.94375228881836, -27.243751525878906, -55....",2
131,CLIMBING,"[[0.2350500077009201, 3.6130502223968506, 8.53...","[[-1.0491249561309814, 0.3264249861240387, 1.9...","[[13.443750381469727, -7.256250381469727, -31....",2
134,CLIMBING,"[[2.2360501289367676, -0.8350500464439392, 9.6...","[[-0.1732500046491623, -0.47795000672340393, -...","[[11.456250190734863, 15.112500190734863, -73....",2
...,...,...,...,...,...
99,CLIMBING,"[[0.48900002241134644, 5.356050491333008, -8.2...","[[0.1909874975681305, 0.5138375163078308, 0.23...","[[-15.412500381469727, -39.95625305175781, 12....",2
92,CLIMBING,"[[1.204050064086914, 6.133950233459473, -5.850...","[[0.15152500569820404, 0.6107749938964844, -0....","[[-2.7562501430511475, -39.84375, -7.143750190...",2
79,CLIMBING,"[[4.345950126647949, 4.465050220489502, -8.359...","[[1.766737461090088, -0.9711624979972839, 1.08...","[[-2.5875000953674316, -41.51250076293945, 3.5...",2
95,CLIMBING,"[[1.8390001058578491, 5.217000484466553, -8.75...","[[-0.027499999850988388, -0.284074991941452, -...","[[-21.543750762939453, -36.30000305175781, 17....",2


In [230]:
def take_first_x_rows_per_category(data_frame_input, category_column, x):
    same_amount_df = pd.DataFrame(columns=data_frame_input.columns)

    # Pętla po unikalnych kategoriach
    for category in data_frame_input[category_column].unique():
        # Wybieranie pierwszych X wierszy z danej kategorii
        selected_rows = data_frame_input[data_frame_input[category_column] == category].head(x)

        # Dodawanie wybranych wierszy do nowego DataFrame'u
        same_amount_df= pd.concat([same_amount_df, selected_rows], ignore_index=True)

    return same_amount_df

In [231]:
same_amount_df = take_first_x_rows_per_category(sorted_df, 'Category', minimum)
same_amount_df

,Category,Accelerometer,Gyroscope,Magnetometer,Duration
0,CLIMBING,"[[0.7330500483512878, 0.4930500090122223, 10.0...","[[0.5670499801635742, -0.09542500227689743, 0....","[[-129.78750610351562, 333.7125244140625, 301....",2
1,CLIMBING,"[[-0.3169500231742859, 1.4239500761032104, 12....","[[0.1942874938249588, -0.2116124927997589, 0.2...","[[39.84375, 63.95625305175781, 10.612500190734...",2
2,CLIMBING,"[[0.3540000021457672, 3.427950143814087, 9.400...","[[-0.9094250202178955, -2.147200107574463, -0....","[[32.94375228881836, -27.243751525878906, -55....",2
3,CLIMBING,"[[0.2350500077009201, 3.6130502223968506, 8.53...","[[-1.0491249561309814, 0.3264249861240387, 1.9...","[[13.443750381469727, -7.256250381469727, -31....",2
4,CLIMBING,"[[2.2360501289367676, -0.8350500464439392, 9.6...","[[-0.1732500046491623, -0.47795000672340393, -...","[[11.456250190734863, 15.112500190734863, -73....",2
...,...,...,...,...,...
207,DOWNTIME,"[[-5.512950420379639, 7.783050537109375, 1.072...","[[-0.013749999925494194, 0.04193750023841858, ...","[[32.70000076293945, -36.65625, 10.08750057220...",2
208,DOWNTIME,"[[-5.794950485229492, 7.659000396728516, 0.903...","[[0.011824999935925007, 0.0013749999925494194,...","[[32.88750076293945, -36.1875, 11.34375], [33....",2
209,DOWNTIME,"[[-5.656050205230713, 7.7470502853393555, 0.90...","[[0.00013749999925494194, 0.037675000727176666...","[[32.88750076293945, -36.54375076293945, 11.34...",2
210,DOWNTIME,"[[-5.664000034332275, 7.711050510406494, 0.691...","[[-0.006874999962747097, 0.011137500405311584,...","[[32.94375228881836, -36.35625076293945, 12.41...",2


In [232]:
same_amount_df.to_csv(f'prepared_data/back_{new_duration}_seconds_normalized.csv')